### Test Case to Cover All Scenarios

To devise a test case that covers all salary scenarios for a hospital nurse, we will utilize detailed entries that encapsulate regular shifts, overtime, night differential, and national holiday pay, while factoring in break times as specified.

#### Specific Scenario Breakdown
- *Regular Hours Calculation*: First 9 hours per shift are paid at the standard rate.
- *Overtime Calculation*: 
  - Additional 20% for overtime up to 50 hours each week.
  - Additional 15% for overtime above 50 hours each week.
  - Additional 25% between 2 AM to 5 AM.
  - Additional 30% on a national holiday.
- *Break Times*: After every 3.5 hours, a 30-minute break is deducted from the work hours calculation.

---

### Automating Payroll Calculation: Algorithm/Pseudocode

This algorithm is designed to automatically calculate the nurse's paycheck, factored by regular hours, various types of overtime, and compliance with break rules:

```
Function CalculatePaycheck(timeLogs):
    totalPay = 0
    weeklyHours = 0

    For each day in timeLogs:
        startTime = Parse time in
        endTime = Parse time out
        totalHours = endTime - startTime
        breaks = totalHours / 3.5 hours
        totalWorkedHours = totalHours - (breaks x 0.5 hours)

        regularHours = Min(totalWorkedHours, 9 hours)
        overtimeHours = totalWorkedHours - 9 hours

        If day is National Holiday:
            rateMultiplier = 1.3
        Else If time between 2 AM to 5 AM:
            rateMultiplier = 1.25
        Else:
            If weeklyHours <= 50:
                rateMultiplier = 1.2
            Else:
                rateMultiplier = 1.15

        dailyPay = (regularHours x $100) + (overtimeHours x $100 x rateMultiplier)
        
        totalPay += dailyPay
        weeklyHours += totalWorkedHours

    Return totalPay
```


This algorithm accommodates breaks, differentiates pay based on the time of day, and keeps a weekly tally to adjust overtime rates accordingly. It could be implemented in programs like Excel using VBA, Python for general scripting, or dedicated payroll software with customizable rules.

In [1]:
# Read the file : test_case.csv and remove first row and again save the file
def modify_file(input, output):
    with open(input, 'r') as file:
        lines = file.readlines()
        with open(output, 'w') as file:
            file.writelines(lines[1:])
    print("File modified successfully")

modify_file('test_case.csv', 'test_case_updated.csv')

File modified successfully


In [2]:
import pandas as pd

data = pd.read_csv('test_case_updated.csv')
data['Event']

0                  NaN
1                  NaN
2                  NaN
3                  NaN
4                  NaN
5     National Holiday
6                  NaN
7                  NaN
8                  NaN
9                  NaN
10                 NaN
11                 NaN
12                 NaN
13                 NaN
14                 NaN
15                 NaN
16    National Holiday
17                 NaN
18                 NaN
19                 NaN
20                 NaN
Name: Event, dtype: object

In [3]:
def convert_time(time : str) -> str:
    if 'AM' in time or 'PM' in time:
        hour, minute = map(int, time[:-3].split(':'))
        if 'PM' in time and hour != 12:
            hour += 12
        if 'AM' in time and hour == 12:
            hour = 0
        return '{:02d}:{:02d}'.format(hour, minute)
    else:
        raise ValueError("Invalid time format. Time should be in 12-hour format with 'AM' or 'PM'.")

data['Time In'] = data['Time In'].apply(convert_time)
data['Time Out same day or next day'] = data['Time Out same day or next day'].apply(convert_time)

data

,Date,Time In,Time Out same day or next day,Event
0,8-Apr-24,09:00,15:00,NaN
1,9-Apr-24,14:00,22:00,NaN
2,10-Apr-24,00:00,13:00,NaN
3,11-Apr-24,03:12,14:48,NaN
4,12-Apr-24,08:10,17:22,NaN
5,13-Apr-24,10:30,13:30,National Holiday
6,14-Apr-24,09:02,20:40,NaN
7,15-Apr-24,22:00,19:36,NaN
8,16-Apr-24,05:00,13:30,NaN
9,17-Apr-24,14:46,22:30,NaN


In [15]:
def calculate_hours(time_in : str, time_out : str) -> float:
    hour_in, minute_in = map(int, time_in.split(':'))
    hour_out, minute_out = map(int, time_out.split(':'))
    hours = (hour_out + minute_out / 60) - (hour_in + minute_in / 60)
    if hours < 0:
        hours += 24
    return hours

def is_conciding_time(time_in, time_out):
    hour_in, minute_in = map(int, time_in.split(':'))
    hour_out, minute_out = map(int, time_out.split(':'))
    if (hour_in >= 2 and hour_in <= 5) or (hour_out >= 2 and hour_out <= 5):
        return True
    return False

def conciding_time(time_in, time_out):
    hour_in, minute_in = map(int, time_in.split(':'))
    hour_out, minute_out = map(int, time_out.split(':'))
    if hour_in >= 2 and hour_in <= 5:
        return min(5, hour_out) - hour_in + (minute_out - minute_in) / 60
    else:
        return hour_out - max(2, hour_in) + (minute_out - minute_in) / 60
    

n = len(data)
hours = 0
start_day = data['Date'][0]
for index, row in data.iterrows():
    time_in = row['Time In']
    time_out = row['Time Out same day or next day']
    hours += calculate_hours(time_in, time_out)
    if (index != 0 and (index + 1) % 7 == 0) or (index == n - 1):
        end_day = row['Date']
        print('Total hours worked for week from {} to {} is {:.2f}\n'.format(start_day, end_day, hours))
        hours = 0
        if index != n - 1:
            start_day = data.iloc[index + 1]['Date']
            
        # Now calulate the total pay for the week
        pay = 0
        for i in range(index - 6, index + 1):
            time_in = data.iloc[i]['Time In']
            time_out = data.iloc[i]['Time Out same day or next day']
            total_hours = calculate_hours(time_in, time_out)
            breaks = total_hours / 3.5
            total_worked_hours = total_hours - (breaks * 0.5)
            regular_hours = min(total_worked_hours, 9)
            overtime_hours = max(total_worked_hours - 9, 0)
            
            print("Total work hours for day {} is {:.2f}".format(data.iloc[i]['Date'], total_worked_hours))
            print("Regular hours for day {} is {:.2f}".format(data.iloc[i]['Date'], regular_hours))
            print("Overtime hours for day {} is {:.2f}".format(data.iloc[i]['Date'], overtime_hours))
            
            day_pay = 0

            if row['Event'] == 'National Holiday':
                day_pay = (regular_hours * 100) + (overtime_hours * 100 * 1.3)
            elif is_conciding_time(time_in, time_out):
                # Get number of hours that nurse has worked between 2 to 5
                conciding_hours = conciding_time(time_in, time_out)
                extra_hours = max(0, overtime_hours - conciding_hours)
                day_pay = (regular_hours * 100) + (conciding_hours * 100 * 1.25) + (extra_hours * 100 * (hours <= 50) * 1.2 + (hours > 50) * 1.15)
            else:
                day_pay = (regular_hours * 100) + (overtime_hours * 100 * (hours <= 50) * 1.2 + (hours > 50) * 1.15)
                
            print("Pay for day {} is {:.2f}\n".format(data.iloc[i]['Date'], day_pay))
            pay += day_pay
            

print("Total pay for the week from {} to {} is {:.2f}".format(start_day, end_day, pay))
                 
    

Total hours worked for week from 8-Apr-24 to 14-Apr-24 is 62.43

Total work hours for day 8-Apr-24 is 5.14
Regular hours for day 8-Apr-24 is 5.14
Overtime hours for day 8-Apr-24 is 0.00
Pay for day 8-Apr-24 is 514.29

Total work hours for day 9-Apr-24 is 6.86
Regular hours for day 9-Apr-24 is 6.86
Overtime hours for day 9-Apr-24 is 0.00
Pay for day 9-Apr-24 is 685.71

Total work hours for day 10-Apr-24 is 11.14
Regular hours for day 10-Apr-24 is 9.00
Overtime hours for day 10-Apr-24 is 2.14
Pay for day 10-Apr-24 is 1157.14

Total work hours for day 11-Apr-24 is 9.94
Regular hours for day 11-Apr-24 is 9.00
Overtime hours for day 11-Apr-24 is 0.94
Pay for day 11-Apr-24 is 1225.00

Total work hours for day 12-Apr-24 is 7.89
Regular hours for day 12-Apr-24 is 7.89
Overtime hours for day 12-Apr-24 is 0.00
Pay for day 12-Apr-24 is 788.57

Total work hours for day 13-Apr-24 is 2.57
Regular hours for day 13-Apr-24 is 2.57
Overtime hours for day 13-Apr-24 is 0.00
Pay for day 13-Apr-24 is 257.14